In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('../data/raw/market_sales.csv',
                     header=0,
                     names=['period', 'user_id', 'store_id', 
                            'item_id', 'license', 'type_by_nomenclature',
                            'rating'],
                     dtype={'user_id': str,
                            'store_id': np.str,
                            'item_id': np.str,
                            'license': np.int8,
                            'type_by_nomenclature': np.str,
                            'rating': np.int32})

In [3]:
raw_data.head()

,period,user_id,store_id,item_id,license,type_by_nomenclature,rating
0,2019-02-01 19:58:36,3000004608438,IZ-000034,00047364,0,C,3
1,2019-02-01 19:58:36,3000004608438,IZ-000034,00152527,0,C,3
2,2019-02-01 19:58:36,3000004608438,IZ-000034,00152528,0,C,3
3,2019-02-01 19:58:36,3000004572289,IZ-000034,00041576,1,A,4
4,2019-02-01 19:58:36,3000003024611,IZ-000034,00041577,1,B,4


In [4]:
cols = raw_data.columns

In [5]:
# deleting spases in string columns
raw_data['user_id'] = raw_data['user_id'].str.strip()
raw_data['store_id'] = raw_data['store_id'].str.strip()
raw_data['item_id'] = raw_data['item_id'].str.strip()

In [6]:
raw_data['user_id'].nunique()

226663

In [7]:
raw_data['store_id'].nunique()

63

In [8]:
raw_data['item_id'].nunique()

45090

In [9]:
raw_data[raw_data.license == 1]['user_id'].nunique()

146672

In [10]:
stores = raw_data.groupby(['store_id'])['item_id'].nunique().reset_index()
stores = stores.rename(columns={'item_id':'unique_goods'})
stores['first_visit_date'] = stores.store_id.apply(lambda x: 
                                                   raw_data[raw_data.store_id == x]
                                                   .period.min())

In [12]:
stores.sort_values('unique_goods')

,store_id,unique_goods,first_visit_date
58,IZ-000073,100,2022-04-27 20:01:23
61,IZ-000082,314,2022-01-04 17:04:10
59,IZ-000076,324,2022-01-04 19:12:13
62,IZ-000083,683,2022-04-13 19:01:05
56,IZ-000070,961,2022-01-03 13:00:21
...,...,...,...
4,IZ-000005,12386,2019-01-02 18:53:34
42,IZ-000053,12590,2019-01-02 20:00:57
3,IZ-000004,12608,2019-01-02 19:05:12
2,IZ-000003,13079,2019-01-02 19:00:21


In [13]:
raw_data.user_id.str.len().value_counts()

13    2490252
15       1994
18       1014
11        659
14        511
17        360
8         329
21        169
12        167
16        109
1          73
24         64
22         11
74          7
Name: user_id, dtype: int64

In [14]:
raw_data[raw_data.user_id.str.len()==13].user_id.unique()

array(['3000004608438', '3000004572289', '3000003024611', ...,
       '3000004633546', '3000006376458', '3000003957797'], dtype=object)

In [15]:
raw_data[raw_data.user_id.str.len()==13]

,period,user_id,store_id,item_id,license,type_by_nomenclature,rating
0,2019-02-01 19:58:36,3000004608438,IZ-000034,00047364,0,C,3
1,2019-02-01 19:58:36,3000004608438,IZ-000034,00152527,0,C,3
2,2019-02-01 19:58:36,3000004608438,IZ-000034,00152528,0,C,3
3,2019-02-01 19:58:36,3000004572289,IZ-000034,00041576,1,A,4
4,2019-02-01 19:58:36,3000003024611,IZ-000034,00041577,1,B,4
...,...,...,...,...,...,...,...
2495714,2022-04-30 19:57:26,3000004472121,IZ-000035,00023780,0,B,3
2495715,2022-04-30 19:57:26,3000004472121,IZ-000035,00001039,0,B,3
2495716,2022-04-30 19:57:26,3000004472121,IZ-000035,00001040,0,B,3
2495717,2022-04-30 19:57:26,3000004472121,IZ-000035,00175960,0,B,4


## Get clients data

In [16]:
def get_client_data(purchases_clear):
    clients = purchases_clear.groupby('user_id').agg({'item_id':'count',
                                                      'period':'min', 
                                                      'license':'max'}).reset_index()
    clients = clients.rename(columns={'user_id':'user_id', 
                                      'item_id':'num_purchases', 
                                      'period':'first_visit'})    
    return clients

In [17]:
# delete strange user_id   
purchases_clear = raw_data[raw_data.user_id.str.len()==13]
# get clients data
clients = get_client_data(purchases_clear)

In [18]:
purchases_clear.user_id.nunique()

226557

In [19]:
purchases_clear.store_id.nunique()

63

In [20]:
purchases_clear.item_id.nunique()

45072

In [21]:
clients[clients.num_purchases > 1000].shape

(34, 4)

In [22]:
purchases_clear[purchases_clear.user_id == '3000000458563'].groupby(['item_id']).count()

,period,user_id,store_id,license,type_by_nomenclature,rating
item_id,,,,,,
00000006,4,4,4,4,4,4
00000011,30,30,30,30,30,30
00000051,1,1,1,1,1,1
00000057,2,2,2,2,2,2
00000059,2,2,2,2,2,2
...,...,...,...,...,...,...
00225365,1,1,1,1,1,1
00225513,3,3,3,3,3,3
00225515,1,1,1,1,1,1


## Get goods info

In [36]:
goods = purchases_clear.groupby('item_id').agg({'license':'max', 
                                        'type_by_nomenclature':'max', 
                                        'rating':'mean'})

In [1]:
purchases_clear['year'] = purchases_clear.period.dt.year()

NameError: name 'purchases_clear' is not defined